<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: преподаватель Факультета Компьютерных Наук НИУ ВШЭ <br> Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание 6. Линейная регрессия, Lasso и RF-регрессия в задаче по определению качества вина</center>

In [22]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**Будем работать с набором данных по качеству белого вина (репозиторий UCI).**
**Загружаем данные.**

In [3]:
data = pd.read_csv('../../data/winequality-white.csv', sep=';')

In [4]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=7) и отмасштабируйте данные с помощью StandardScaler.**

In [6]:
y = data.quality
data.drop(['quality'], axis=1, inplace=True)

X_train, X_holdout, y_train, y_holdout = train_test_split(data, y, test_size=0.3, random_state=7)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Линейная регрессия

**Обучите простую линейную регрессию.**

In [9]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

**Оцените среднеквадратичную ошибку модели на обучающей и отложенной выборках.**

In [80]:
def quality(model):
    predictions_test = model.predict(X_holdout_scaled)
    predictions_train = model.predict(X_train_scaled)
    
    print("Среднеквадратическа ошибка train: {}".format(round(mean_squared_error(predictions_train, y_train), 3)))
    print("Среднеквадратическа ошибка test: {}".format(round(mean_squared_error(predictions_test, y_holdout), 3)))

In [81]:
quality(linreg)

Среднеквадратическа ошибка train: 0.551
Среднеквадратическа ошибка test: 0.593


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.**

In [65]:
linreg_coef = pd.DataFrame(linreg.coef_, X_train.columns)
linreg_coef.columns = ["coef"]
linreg_coef["abs"] = linreg_coef.coef.apply(np.abs)
linreg_coef.sort_values(by="abs", ascending=False).drop(["abs"], axis=1)

,coef
density,-0.397251
residual sugar,0.374399
alcohol,0.264881
volatile acidity,-0.188744
pH,0.093467
sulphates,0.063989
free sulfur dioxide,0.053009
fixed acidity,0.042850
total sulfur dioxide,-0.023227
chlorides,-0.004759


Плотность с сахаром и алкоголем, согласно линейной регрессии, влияют сильнее всего, посмотрим, так ли это в остальных моделях

## Lasso-регрессия

**Обучите Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=7.**

In [60]:
lasso1 = Lasso(alpha=.01, random_state=7)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=7,
   selection='cyclic', tol=0.0001, warm_start=False)

**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**

In [67]:
lasso1_coef = pd.DataFrame(lasso1.coef_, X_train.columns)
lasso1_coef.columns = ["coef"]
lasso1_coef["abs"] = lasso1_coef.coef.apply(np.abs)
lasso1_coef.sort_values(by="abs", ascending=False).drop(["abs"], axis=1)

,coef
alcohol,0.377739
volatile acidity,-0.187848
residual sugar,0.175769
density,-0.122757
free sulfur dioxide,0.042205
sulphates,0.036801
pH,0.034998
fixed acidity,-0.012489
chlorides,-0.011151
total sulfur dioxide,-0.010105


Что ж, кислота отвалилась, как наименее важная, к тому же у нас новые фавориты - на место плотности и сахара вышли алкоголь и переменная кислотность

**Теперь определите лучшее значение $\alpha$ в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=7.**

In [68]:
alphas = np.linspace(0.1, 10, 200)
lasso_cv = LassoCV(alphas=alphas, random_state=7, cv=5)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([  0.1    ,   0.14975, ...,   9.95025,  10.     ]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=7, selection='cyclic', tol=0.0001,
    verbose=False)

In [70]:
print("Лучшее значение alpha: {}".format(round(lasso_cv.alpha_, 3)))

Лучшее значение alpha: 0.1


**Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина. Какие 2 признака оказались главными?**

In [72]:
lasso_cv_coef = pd.DataFrame(lasso_cv.coef_, X_train.columns)
lasso_cv_coef.columns = ["coef"]
lasso_cv_coef["abs"] = lasso_cv_coef.coef.apply(np.abs)
lasso_cv_coef.sort_values(by="abs", ascending=False).drop(["abs"], axis=1)

,coef
alcohol,0.300389
volatile acidity,-0.090348
fixed acidity,-0.000000
citric acid,0.000000
residual sugar,0.000000
chlorides,-0.000000
free sulfur dioxide,0.000000
total sulfur dioxide,-0.000000
density,-0.000000
pH,0.000000


С таким высоким `alpha`, только две переменных оказались значимыми - алкоголь и переменная кислотность

**Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.**

In [82]:
quality(lasso_cv)

Среднеквадратическа ошибка train: 0.599
Среднеквадратическа ошибка test: 0.657


Избавившись от кучи переменных мы, конечно, получили худшее качество, чем было раньше, но зато можно надеяться, что результаты будут устойчивее

## Случайный лес

**Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=7.**

In [84]:
forest = RandomForestRegressor(random_state=7)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=7,
           verbose=0, warm_start=False)

**Оцените среднеквадратичную ошибку модели на обучающей выборке, на кросс-валидации (cross_val_score с scoring='mean_squared_error' и остальными параметрами по умолчанию) и на оставленной выборке.**

In [85]:
quality(forest)

Среднеквадратическа ошибка train: 0.078
Среднеквадратическа ошибка test: 0.446


Всего лишь при помощи 10 деревьев лес сумел прекрасно подогнать себя под обучающую выборку, хотя и на тестовой качество оказалось лучше, чем у всех моделей до него

In [94]:
print("Среднеквадратическая ошибка на кросс-валидации: {}".format(round(
        np.mean(np.abs(
                cross_val_score(forest, X_train_scaled, y_train, scoring="mean_squared_error"))), 3
               )
        )
     )

Среднеквадратическая ошибка на кросс-валидации: 0.458


На кросс-валидации ошибка не сильно отличается от отложенной выборки, так что действительно наше качество леса лучше, чем у предыдущих моделей

**Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на оставленной выборке.**

In [95]:
forest_params = {'max_depth': list(range(15, 20)), 
                 'min_samples_leaf': list(range(2, 8))}

locally_best_forest = GridSearchCV(RandomForestRegressor(n_jobs=-1, random_state=7), 
                                 forest_params, 
                                 scoring='mean_squared_error',  
                                 n_jobs=-1, cv=5)
locally_best_forest.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=-1, oob_score=False, random_state=7,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [15, 16, 17, 18, 19], 'min_samples_leaf': [2, 3, 4, 5, 6, 7]},
       pre_dispatch='2*n_jobs', refit=True, scoring='mean_squared_error',
       verbose=0)

In [102]:
locally_best_forest.best_params_

{'max_depth': 17, 'min_samples_leaf': 3}

В результате грид серча на кросс-валидации мы получили чуть более жесткие требования для нашего дерева, вместо 1 наблюдения для листа нам теперь нужно 3, плюс ограничили максимальную глубину, что подняло ошибку на train-датасете и, похоже, снизило вероятность переобучения

In [100]:
quality(locally_best_forest)

Среднеквадратическа ошибка train: 0.15
Среднеквадратическа ошибка test: 0.453


In [101]:
print("Среднеквадратическая ошибка на кросс-валидации: {}".format(round(
        np.mean(np.abs(
                cross_val_score(locally_best_forest, X_train_scaled, y_train, scoring="mean_squared_error"))), 3
               )
        )
     )

Среднеквадратическая ошибка на кросс-валидации: 0.469


**Оцените важность признаков с помощью случайного леса.**

In [112]:
rf_importance = pd.DataFrame(forest.fit(X_train_scaled, y_train).feature_importances_, X_train.columns)
rf_importance.sort_values(0, ascending=False)

,0
alcohol,0.256848
volatile acidity,0.117998
free sulfur dioxide,0.112029
total sulfur dioxide,0.077374
pH,0.073050
chlorides,0.064139
residual sugar,0.062574
fixed acidity,0.060147
density,0.059914
sulphates,0.059120


**Сделайте выводы о качестве моделей и оценках влияния признаков на качество вина с помощью этих трех моделей.**

- Линейная регрессия дала неплохое baseline решение, если судить по метрике качества - среднеквадратической ошибке, то она даже чуть точнее, чем лассо регрессия, однако дело всё в количестве и в качестве переменных - линейная регрессия на первые места поставила далеко не самые важные признаки, что, вполне вероятно, привело к переобучению. 

- Лассо регрессия от таких переменных успешно избавилась, что неибежно повлекло за собой рост среднеквадратической ошибки, но зато дало более точное представление о действительно важных признаках, влияющих на качество вина. В результате при относительно небольшой потере качества (+0.06 к mse на валидации), мы получили гораздо более простую модель, состоящую всего из двух признаков - алкоголя, который положительно влияет на оценку качества вина, и переменной кислотности, влияющей отрицательно. 

- Наконец, случайный лес. Без предварительного подбора параметров мы получили идеально подогнонанную под обучающую выборку модель, которая, тем не менее, давала заметно лучшие результаты и на тестовом датасете. После подбора параметров разница в качестве на тестовом и на обучающем датасетах была снижена, при этом на тесте mse подрос лишь на 0.01, а мы получили более устойчивую модель. При этом выбранные по лассо регрессии переменные оказались самыми важными и по мнению случайного леса, значит, действительно эти две переменные являются определяющими. 